In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
# Load Data
train_data_df = pd.read_csv(r'C:\Users\%%userprofile%%\Downloads\playground-series-s4e8\train.csv')
train_data_df.head()


,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [3]:

test_data_df = pd.read_csv(r'C:\Users\%%userprofile%%\Downloads\playground-series-s4e8\test.csv')

In [4]:

testdatanew = pd.read_csv(r'C:\Users\%%userprofile%%\Downloads\playground-series-s4e8\test.csv')

In [5]:
labelEncoder = LabelEncoder()
standardScaler = StandardScaler()

In [6]:
train_data_df[train_data_df.select_dtypes(include=['object']).columns] = train_data_df.select_dtypes(include=['object']).apply(lambda x: x.astype('category')).apply(labelEncoder.fit_transform)

test_data_df[test_data_df.select_dtypes(include=['object']).columns] = test_data_df.select_dtypes(include=['object']).apply(lambda x: x.astype('category')).apply(labelEncoder.fit_transform)


In [7]:
y_train = train_data_df['class']

In [8]:
y_train

0          0
1          1
2          0
3          0
4          0
          ..
3116940    0
3116941    0
3116942    1
3116943    0
3116944    1
Name: class, Length: 3116945, dtype: int32

In [9]:
train_data_df[train_data_df.select_dtypes(exclude=['object']).columns] = standardScaler.fit_transform(train_data_df.select_dtypes(exclude=['object']))
test_data_df[test_data_df.select_dtypes(exclude=['object']).columns] = standardScaler.fit_transform(test_data_df.select_dtypes(exclude=['object']))

In [10]:
# 2. Separate features (X) and target (y)
X_train = train_data_df.drop(['class','id'], axis=1) 


X_test = test_data_df.drop('id',axis=1)




In [11]:
'''
X_lda_train = X_train.fillna(0)
X_lda_test = X_test.fillna(0)
# 4. Apply LDA
lda = LinearDiscriminantAnalysis(n_components=1) 
lda.fit(X_lda_train, y_train)
'''

from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Fill missing values in the training data
X_lda_train = imputer.fit_transform(X_train)

# Fill missing values in the test data
X_lda_test = imputer.transform(X_test)

# Apply LDA
#lda = LogisticRegression()
#lda.fit(X_lda_train, y_train)

rf_model = RandomForestClassifier(n_estimators=2, random_state=42)
rf_model.fit(X_lda_train, y_train)



RandomForestClassifier(n_estimators=2, random_state=42)

In [13]:
# pridction 

y_pred = rf_model.predict(X_lda_test)
y_pred_df = pd.DataFrame(y_pred,columns=['class'])


final = pd.concat([testdatanew['id'],y_pred_df],axis=1)

final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077964 entries, 0 to 2077963
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   id      int64
 1   class   int32
dtypes: int32(1), int64(1)
memory usage: 23.8 MB


In [14]:
final['class'] = final['class'].apply(lambda x: 'e' if x == 0 else 'p')
final

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [15]:
value_counts = final['class'].value_counts()

# Display the counts
print(value_counts)

class
e    1073970
p    1003994
Name: count, dtype: int64


In [16]:
final.to_csv(r'C:\Users\%%userprofile%%\Downloads\playground-series-s4e8\pridction.csv', index=False)